In [1]:
import os
import re
import csv
from PIL import Image
import pandas as pd
import torch
import torchvision.models as models
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )])

def get_vector(input_image):
    image = input_image.convert("RGB")  # in case input image is not in RGB format
    img_t = transform(image)
    batch_t = torch.unsqueeze(img_t, 0)
    my_embedding = torch.zeros([1, 512, 1, 1])
    def copy_data(m, i, o):
        my_embedding.copy_(o.data)
    h = layer.register_forward_hook(copy_data)
    model(batch_t)
    h.remove()
    return my_embedding.squeeze().cpu().numpy()

model = models.resnet18(pretrained=True)
layer = model._modules.get('avgpool')
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [2]:
im_path = 'C:/Users/subha/Desktop/Scientific Project IR/smalltest'
im_names = [os.path.join(root, name)
            for root, dirs, files in os.walk(im_path)
            for name in files
            if name.endswith(".jpg")]
existing_images_df = pd.DataFrame([re.findall(r"[\w']+", im_name)[1:3] for im_name in im_names],
                                  columns=['cat_id', 'pid'])
existing_images_df['impath'] = im_names
vecs = [list(get_vector(Image.open(impath))) for _, pid, impath in existing_images_df.values]
with open('C:/Users/subha/Desktop/Scientific Project IR/smalltest.tsv', 'w') as fw:
    csv_writer = csv.writer(fw, delimiter='\t')
    csv_writer.writerows(vecs)

In [3]:
from scipy import spatial
from sklearn.metrics.pairwise import cosine_similarity

pic_one = Image.open("C:/Users/subha/Desktop/Scientific Project IR/smalltest/2007_000243.jpg")
a=get_vector(pic_one)
directory = r'C:/Users/subha/Desktop/Scientific Project IR/smalltest'
for filename in os.listdir(directory):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        pic_two = Image.open(os.path.join(directory, filename))
        b = get_vector(pic_two)
        result = 1 - spatial.distance.cosine(a, b)
        print(str(filename) + " , " + str(result))      
    else:
        continue

        
# from scipy import spatial


# result = 1 - spatial.distance.cosine(a, b)
# print(result)

# from sklearn.metrics.pairwise import cosine_similarity
# cosine_similarity(a.reshape(1,-1), b.reshape(1,-1))

2007_000027.jpg , 0.4329763352870941
2007_000243.jpg , 1.0
2007_000648.jpg , 0.4924065172672272
2007_000664.jpg , 0.3572952151298523
2007_000676.jpg , 0.43987682461738586
2007_000720.jpg , 0.373544842004776
2007_000733.jpg , 0.3809690773487091
2007_000738.jpg , 0.592597484588623
2007_000768.jpg , 0.3978719711303711
2007_000822.jpg , 0.4634544551372528
2007_000999.jpg , 0.4409947395324707
